<a href="https://colab.research.google.com/github/sabbhy/CSE573-StanceDetection/blob/main/DataSetPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.5 MB/s 


In [3]:
from google.colab import files
import json
import os
import pandas as pd
import string
import re
import demoji

In [4]:
df = pd.DataFrame(columns=['id', 'parent_id', 'tweet', 'topic'])
print(df)

Empty DataFrame
Columns: [id, parent_id, tweet, topic]
Index: []


In [5]:
# file = open('/content/drive/CSE 573 Semantic Web Mining/processed/charliehebdo/552783667052167168/source-tweet/552783667052167168.json')
# jsonfile = json.load(file)
# print(jsonfile)
pd.options.display.max_colwidth = 10000;
dir = '/content/drive/Shareddrives/CSE 573 Semantic Web Mining/processed/'
for topic in os.listdir(dir):
  # print(topic)
  for thread in os.listdir(dir+topic+'/'):
    # print(thread)
    threadDir = dir+topic+'/'+thread+'/'
    if(os.path.exists(threadDir+'source-tweet/')):
      for sourceTweet in os.listdir(threadDir+'source-tweet/'):
        file = open(threadDir + 'source-tweet/'+sourceTweet)
        jsonfile = json.load(file)
        df.loc[len(df.index)] = [jsonfile['id'], jsonfile['in_reply_to_status_id'], jsonfile['text'], topic]
    if(os.path.exists(threadDir+'replies/')):
      for reply in os.listdir(threadDir+'replies/'):
        file = open(threadDir + 'replies/'+reply)
        jsonfile = json.load(file)
        # print(jsonfile['text'])
        df.loc[len(df.index)] = [jsonfile['id'], jsonfile['in_reply_to_status_id'], jsonfile['text'], topic]




In [6]:
display(df['tweet'][0])

'Store owner told @FOX2now Monday that there was a theft, but said it was not MichaelBrown; said it was someone else #Ferguson'

In [7]:
pd.options.display.max_colwidth = 10000;
df_train = pd.DataFrame(columns=['id', 'parent_id', 'parent_tweet', 'tweet', 'topic', 'classification'])
df_train_body = pd.DataFrame(columns=['id', 'tweet'])
df_train_stance = pd.DataFrame(columns=['parent_tweet', 'id', 'classification'])


In [8]:
for index, row in df.iterrows():
  row['tweet'] = " ".join(filter(lambda word: not word.startswith('@'), row['tweet'].split()))
  row['tweet'] = " ".join(filter(lambda word: not word.startswith('http'), row['tweet'].split()))
  row['tweet'] = "".join([char for char in row['tweet'] if char not in ['#','@']])
  row['tweet'] = re.sub('[0-9]+', '', row['tweet'])
  row['tweet'] = demoji.replace(row['tweet'], "")


In [ ]:
from pandas.core.arrays.sparse import dtype
trainJsonPath = '/content/drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/rumoureval-subtaskA-train.json'
trainfile = open(trainJsonPath)
trainjsonfile = json.load(trainfile)
for tweet_id, annotation in trainjsonfile.items():
  row = df.loc[df['id'] == int(tweet_id)]
  parentid = row['parent_id'].to_string(index=False)
  parentTweet = None
  if(parentid not in ["None", "Series([], )"]):
    parentTweet = df[df['id'] == int(parentid)]['tweet'].to_string(index=False)
  df_train.loc[len(df_train.index)] = [row['id'].to_string(index=False), row['parent_id'].to_string(index=False), parentTweet, row['tweet'].to_string(index=False), row['topic'].to_string(index=False), annotation]
  if(parentTweet != None):
    df_train_body.loc[len(df_train_body.index)] =  [row['id'].to_string(index=False), row['tweet'].to_string(index=False)]
    df_train_stance.loc[len(df_train_stance.index)] = [parentTweet, row['id'].to_string(index=False), annotation]



In [10]:
display(df_train_stance)

,parent_tweet,id,classification
0,"Heard that friend was not going to be ""charged"" in any robbery since video shows it was other person",501760642928635904,comment
1,"Name of Ferguson cop is expected to be released by ET, police chief says: (J.B. Forbes, AP)",500270212198174720,comment
2,"no, radical Muslim extremists are making us victims.",524971210275565568,comment
3,URGENT: gunmen involved in deadly attack on CharlieHebdo - French interior minister Paris,552836882770690049,comment
4,Did anyone think of comparing the clothes that MikeBrown was wearing when he was laying on the street to the security footage? Ferguson,500289931097296897,support
...,...,...,...
3959,CharlieHebdo shooting suspects tell police they 'prepared to die as martyrs' - French media,553505863827079168,comment
3960,"BREAKING NEWS: Gunmen take hostages at Sydney cafe, wave Islamic flag",544284128615473152,comment
3961,"There are pics of ""strong arm robbery"" clothing description/size of assailant similar BUT cop deadly force? No!",500302889429241856,support
3962,"According to Seven, this is a picture of the gunman",544293668513792000,comment


In [11]:
df_train.to_csv('train.csv')
df_train_body.to_csv('train_body.csv')
df_train_stance.to_csv('train_stance.csv')

In [12]:
df_dev = pd.DataFrame(columns=['id', 'parent_id', 'tweet', 'topic', 'classification'])
df_dev_body = pd.DataFrame(columns=['id', 'tweet'])
df_dev_stance = pd.DataFrame(columns=['parent_tweet', 'id', 'classification'])

In [13]:
devJsonPath = '/content/drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/rumoureval-subtaskA-dev.json'
devfile = open(devJsonPath)
devjsonfile = json.load(devfile)
for tweet_id, annotation in devjsonfile.items():
  row = df.loc[df['id'] == int(tweet_id)]
  row['classification'] = annotation
  parentid = row['parent_id'].to_string(index=False)
  parentTweet = None
  if(parentid not in ["None", "Series([], )"]):
    parentTweet = df[df['id'] == int(parentid)]['tweet'].to_string(index=False)
  df_dev.loc[len(df_dev.index)] = [row['id'].to_string(index=False), row['parent_id'].to_string(index=False), row['tweet'].to_string(index=False), row['topic'].to_string(index=False), annotation]
  if(parentTweet != None):
    df_dev_body.loc[len(df_dev_body.index)] =  [row['id'].to_string(index=False), row['tweet'].to_string(index=False)]
    df_dev_stance.loc[len(df_dev_stance.index)] = [parentTweet, row['id'].to_string(index=False), annotation]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
display(df_dev_stance)

,parent_tweet,id,classification
0,Germanwings Airbus A crashes in French Alps near Digne,580321716838100992,support
1,U From FL to FL in min? That's average rate of descent fpm! Than levelled off at FL? What the hell was going on there?,580332626541694976,support
2,The transponder did not say anything. No sqwak .,581065934598582272,comment
3,"Latest on Germanwings crash: Pilots signaled before dropping out of midair; airline CEO calls this a ""dark day.""",580342887910494208,query
4,all rumors.not confirmed at all,581576082362478595,comment
...,...,...,...
251,"Debris from Germanwings crash located in Barcelonette area of the Alps, France's Interior Ministry says",580397833548861442,comment
252,GERMAN NEWS: Co-Pilot of Germanwings Airbus Was MUSLIM CONVERT. Can any muslim be trusted near an airplane now?,581338469496041472,comment
253,BREAKING: passengers were on board GermanWings Airbus A which has crashed in the southern French Alps -,580321806285975552,comment
254,BREAKING: feared dead in crashed Germanwings flight,580342470472400897,support


In [14]:
df_dev.to_csv('dev.csv')
df_dev_stance.to_csv('dev_stance.csv')
df_dev_body.to_csv('dev_body.csv')

In [16]:
!cp train.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"
!cp dev.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"
!cp train_body.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"
!cp dev_body.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"
!cp train_stance.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"
!cp dev_stance.csv "drive/Shareddrives/CSE 573 Semantic Web Mining/Annotated/"